## Análisis de redes de la elección a magistrados de salas de apelaciones en el congraso de Guatemala en 2024

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
driver = webdriver.Firefox()

In [3]:
a = 7900
b = 7926

In [7]:
dfs = []
urls = [f"https://www.congreso.gob.gt/detalle_de_votacion/4{i}/41228" for i in range(a, b)]

for url in urls:
    
    driver.get(url)
    
    element_select_favor = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "congreso_a_favor_length"))
    )
    select_favor = Select(element_select_favor)
    select_favor.select_by_value("-1")
    
    nav_link_contra = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, 'CONTRA'))
    )
    nav_link_contra.click()
    
    element_select_contra = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "congreso_contra_length"))
    )
    select_contra = Select(element_select_contra)
    select_contra.select_by_value("-1")
    
    nav_link_nulo = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, 'AUSENCIA'))
    )
    nav_link_nulo.click()
    
    element_select_nulo = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "congreso_votos_nulos_length"))
    )
    select_nulo = Select(element_select_nulo)
    select_nulo.select_by_value("-1")
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    titulo = soup.find('h5', id='title_list')
    parrafos_titulo = titulo.find_all('p')
    pregunta = parrafos_titulo[1]
    candidato = pregunta.text.replace('Pregunta: ELECCIÓN DE ', '').replace(' COMO MAGISTRADO DE LA CORTE SUPREMA DE JUSTICIA', '')
    numero = parrafos_titulo[2]
    numero = numero.text.replace('Número: ', '')
    
    tablas = soup.find_all('table')
    
    df_favor = pd.read_html(str(tablas))[4]
    df_contra = pd.read_html(str(tablas))[5]
    df_nulos = pd.read_html(str(tablas))[6]
    
    df_concatenado = pd.concat([df_favor, df_contra, df_nulos], axis=0)
    df_ordenado = df_concatenado.sort_values(by='NOMBRE DIPUTADO')
    df = df_ordenado.drop(columns=['ESTADO'])
    df = df.replace('Ningún dato disponible en esta tabla =(', np.nan)
    df = df.dropna()
    df['VOTO'] = df['VOTO'].replace({'CONTRA': '0', 'A FAVOR': '1'})
    df = df.rename(columns={'VOTO': candidato})
    df = df.rename(columns={'NOMBRE DIPUTADO': 'DIPUTADO'})
    
    dfs.append(df)
    
    df.to_csv(f'archivos/{numero} {candidato}.csv', index=False)

/tmp/ipykernel_11338/3998319732.py:47: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_favor = pd.read_html(str(tablas))[4]
/tmp/ipykernel_11338/3998319732.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_contra = pd.read_html(str(tablas))[5]
/tmp/ipykernel_11338/3998319732.py:49: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_nulos = pd.read_html(str(tablas))[6]
/tmp/ipykernel_11338/3998319732.py:47: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_favor = pd.read_html(str(tablas))[4]

In [8]:
driver.close()

In [9]:
#Crear una lista de todos los nombre de candidadtos
#Crear una lista con todos los nombres de diputados
candidatos = []
diputados = set()

for df in dfs:
    candidatos.append(df.columns.values[1])
    diputados.update(df['DIPUTADO'].values)
    
candidatos = sorted(candidatos)
diputados = sorted(list(diputados))

In [10]:
# Crear la matriz de adyacencia
matriz_adyacencia = pd.DataFrame(0, index=diputados, columns=candidatos)

In [13]:
# Llenando la matriz de adyacencia con los datos en los dataframes
for df in dfs:
    for i, row in df.iterrows():
        diputado = row['DIPUTADO']
        candidato = df.columns[1]
        valor = row[candidato]
        if valor == '1':
            matriz_adyacencia.loc[diputado, candidato] = 1

matriz_adyacencia.to_csv('archivos/matriz_adyacencia_CSJ.csv')